In [210]:
import re
import os
import numpy as np
import pandas as pd

In [211]:
RAW_PATH = "../data/raw/batdongsan_com_vn.csv"
OUT_PATH = "../data/preprocessed/batdongsancomvn_preprocessed.csv"

df = pd.read_csv(RAW_PATH, encoding = "utf-8")
print(df.shape)
df.head()

(8414, 20)


,property_type,price,area,n_bedrooms,n_bathrooms,n_floors,legal,interior,facing_direction,balcony_direction,front_width,front_road_width,title,latitude,longitude,verified,location,location_details,date_of_posting,url
0,Căn hộ chung cư,"7,7 tỷ",75 m²,2 phòng,NaN,NaN,Sổ đỏ/ Sổ hồng,Đầy đủ,Bắc,Nam,NaN,NaN,Bán căn hộ 2PN tháp T5 dự án Masteri Thảo Điền...,10.802441,106.741950,1.0,"Quận 2, Hồ Chí Minh","Dự án Masteri Thảo Điền, Đường Xa Lộ Hà Nội, P...",16/12/2025,https://batdongsan.com.vn/ban-can-ho-chung-cu-...
1,Nhà riêng,"15,8 tỷ",72 m²,6 phòng,6 phòng,6 tầng,Sổ đỏ/ Sổ hồng,Đầy đủ,NaN,NaN,"6,1 m",5 m,"Bán nhà 6 tầng, thang máy gara 3 ô tô 72m² ngõ...",21.038832,105.934539,1.0,"Long Biên, Hà Nội","Phúc Lợi, Phúc Lợi, Long Biên, Hà Nội",13/12/2025,https://batdongsan.com.vn/ban-nha-rieng-duong-...
2,Nhà riêng,"7,5 tỷ","102,3 m²",2 phòng,1 phòng,2 tầng,Sổ đỏ/ Sổ hồng,Đầy đủ,Đông - Nam,NaN,5 m,8 m,Bán nhà view trực diện Delight Park Đà Lạt khô...,11.973893,108.484595,1.0,"Đà Lạt, Lâm Đồng","Phường 12, Đà Lạt, Lâm Đồng",14/12/2025,https://batdongsan.com.vn/ban-nha-rieng-phuong...
3,Nhà riêng,"6,6 tỷ","60,9 m²",2 phòng,2 phòng,1 tầng,Sổ đỏ/ Sổ hồng,Cơ bản,NaN,NaN,"4,35 m",6 m,Nhà chính chủ Phạm Văn Chiêu- Hẻm 6m thông Lê ...,10.849654,106.651222,1.0,"Gò Vấp, Hồ Chí Minh","Đường Phạm Văn Chiêu, Phường 9, Gò Vấp, Hồ Chí...",13/12/2025,https://batdongsan.com.vn/ban-nha-rieng-duong-...
4,Nhà riêng,59 tỷ,148 m²,41 phòng,41 phòng,10 tầng,Sổ đỏ/ Sổ hồng,Đầy đủ,NaN,NaN,"6,5 m",5 m,Siêu phẩm Đồng Me tòa 10 tầng 41 phòng full dò...,21.011918,105.776275,1.0,"Nam Từ Liêm, Hà Nội","Đường Đồng Me, Phường Mễ Trì, Nam Từ Liêm, Hà Nội",12/12/2025,https://batdongsan.com.vn/ban-nha-rieng-duong-...


In [212]:
df.info()
df.isna().mean().sort_values(ascending = False).head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8414 entries, 0 to 8413
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   property_type      8414 non-null   object 
 1   price              8390 non-null   object 
 2   area               8390 non-null   object 
 3   n_bedrooms         6393 non-null   object 
 4   n_bathrooms        6072 non-null   object 
 5   n_floors           5210 non-null   object 
 6   legal              7590 non-null   object 
 7   interior           5175 non-null   object 
 8   facing_direction   3123 non-null   object 
 9   balcony_direction  2450 non-null   object 
 10  front_width        4353 non-null   object 
 11  front_road_width   3906 non-null   object 
 12  title              8389 non-null   object 
 13  latitude           8389 non-null   float64
 14  longitude          8389 non-null   float64
 15  verified           8414 non-null   float64
 16  location           8413 

balcony_direction    0.708819
facing_direction     0.628833
front_road_width     0.535774
front_width          0.482648
interior             0.384954
n_floors             0.380794
n_bathrooms          0.278346
n_bedrooms           0.240195
legal                0.097932
location_details     0.002971
title                0.002971
latitude             0.002971
longitude            0.002971
date_of_posting      0.002971
price                0.002852
dtype: float64

In [213]:
# drop features
cols_to_drop = [
    "balcony_direction",
    "front_road_width",
    "title",
    "verified",
    "date_of_posting",
    "url", "facing_direction"
]
df = df.drop(columns = [c for c in cols_to_drop if c in df.columns])
print(df.shape)
df.head(3)

(8414, 13)


,property_type,price,area,n_bedrooms,n_bathrooms,n_floors,legal,interior,front_width,latitude,longitude,location,location_details
0,Căn hộ chung cư,"7,7 tỷ",75 m²,2 phòng,NaN,NaN,Sổ đỏ/ Sổ hồng,Đầy đủ,NaN,10.802441,106.741950,"Quận 2, Hồ Chí Minh","Dự án Masteri Thảo Điền, Đường Xa Lộ Hà Nội, P..."
1,Nhà riêng,"15,8 tỷ",72 m²,6 phòng,6 phòng,6 tầng,Sổ đỏ/ Sổ hồng,Đầy đủ,"6,1 m",21.038832,105.934539,"Long Biên, Hà Nội","Phúc Lợi, Phúc Lợi, Long Biên, Hà Nội"
2,Nhà riêng,"7,5 tỷ","102,3 m²",2 phòng,1 phòng,2 tầng,Sổ đỏ/ Sổ hồng,Đầy đủ,5 m,11.973893,108.484595,"Đà Lạt, Lâm Đồng","Phường 12, Đà Lạt, Lâm Đồng"


In [214]:
# rename features


rename_map = {
    "location_details": "address",
}
df = df.rename(columns = {k: v for k, v in rename_map.items() if k in df.columns})
df.columns

Index(['property_type', 'price', 'area', 'n_bedrooms', 'n_bathrooms',
       'n_floors', 'legal', 'interior', 'front_width', 'latitude', 'longitude',
       'location', 'address'],
      dtype='object')

In [230]:
# split location

def split_location(loc: str):
    if pd.isna(loc):
        return pd.Series([np.nan, np.nan])
    parts = [x.strip() for x in str(loc).split(",")]
    if len(parts) >= 2:
        district = parts[0]
        city_province = parts[1]
        return pd.Series([district, city_province])
    return pd.Series([np.nan, np.nan])

df[["district", "city_province"]] = df["location"].apply(split_location)
df = df.drop(columns = ["location"])
df[["district", "city_province"]].head(5)

KeyError: 'location'

In [231]:
def to_float_vn(x):
    """
    Convert Vietnamese number string with comma decimal to float.
    Examples: '7,7' -> 7.7 ; '4,35' -> 4.35
    """
    if pd.isna(x):
        return np.nan
    s = str(x).strip().lower()
    s = s.replace(",", ".")
    try:
        return float(s)
    except:
        return np.nan

def extract_int(x):
    """
    Extract first integer from a string.
    Examples: '2 phòng' -> 2 ; '6 tầng' -> 6
    """
    if pd.isna(x):
        return np.nan
    m = re.search(r"\d+", str(x))
    return float(m.group()) if m else np.nan  # float for NaN compatibility

In [232]:
def clean_area_m2(x):
    if pd.isna(x):
        return np.nan
    s = str(x).lower().strip()
    s = s.replace("m²", "").replace("m2", "").replace(" ", "")
    s = s.replace(",", ".")
    s = re.sub(r"[^0-9.]", "", s)
    try:
        return float(s)
    except:
        return np.nan

df["area"] = df["area"].apply(clean_area_m2)
df["area"].head(5)

0     75.0
1     72.0
2    102.3
3     60.9
4    148.0
Name: area, dtype: float64

In [233]:
for col in ["n_bedrooms", "n_bathrooms", "n_floors"]:
    if col in df.columns:
        df[col] = df[col].apply(extract_int)

df[["n_bedrooms", "n_bathrooms", "n_floors"]].head(5)

,n_bedrooms,n_bathrooms,n_floors
0,2.0,NaN,NaN
1,6.0,6.0,6.0
2,2.0,1.0,2.0
3,2.0,2.0,1.0
4,41.0,41.0,10.0


In [234]:
def clean_width_m(x):
    if pd.isna(x):
        return np.nan
    s = str(x).lower().strip()
    s = s.replace("m", "").replace(" ", "")
    s = s.replace(",", ".")
    s = re.sub(r"[^0-9.]", "", s)
    try:
        return float(s)
    except:
        return np.nan

if "front_width" in df.columns:
    df["front_width"] = df["front_width"].apply(clean_width_m)

df["front_width"].head(5)

0     NaN
1    6.10
2    5.00
3    4.35
4    6.50
Name: front_width, dtype: float64

In [235]:
# -------------------------------
# Clean & standardize price (billion VND)
# Handles:
# - "x tỷ"
# - "y triệu/m²"
# - invalid cases -> NaN
# -------------------------------

def clean_price_billion(row):
    price_raw = row["price"]
    area = row["area"]

    if pd.isna(price_raw):
        return np.nan

    s = str(price_raw).lower().strip()

    # remove spaces
    s = s.replace(" ", "")

    # -------- Case 1: x tỷ --------
    if "tỷ" in s or "ty" in s:
        s = s.replace("tỷ", "").replace("ty", "")
        s = s.replace(",", ".")
        s = re.sub(r"[^0-9.]", "", s)
        try:
            return float(s)
        except:
            return np.nan

    # -------- Case 2: y triệu / m2 --------
    if "triệu/m" in s or "tr/m" in s or "tr/m2" in s or "triệu/m2" in s:
        if pd.isna(area):
            return np.nan

        s = s.replace("triệu/m²", "").replace("triệu/m2", "")
        s = s.replace("tr/m²", "").replace("tr/m2", "")
        s = s.replace(",", ".")
        s = re.sub(r"[^0-9.]", "", s)

        try:
            price_per_m2_million = float(s)
            total_price_billion = (price_per_m2_million * area) / 1000
            return round(total_price_billion, 3)
        except:
            return np.nan

    # -------- Other cases --------
    return np.nan


df["price"] = df.apply(clean_price_billion, axis = 1)

df["price"].head(15)

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
10   NaN
11   NaN
12   NaN
13   NaN
14   NaN
Name: price, dtype: float64

In [236]:
# Inspect listings priced per m²
mask_price_per_m2 = df["price"].isna() & df["area"].notna()

df.loc[mask_price_per_m2, ["price", "area"]].head(10)

,price,area
0,NaN,75.0
1,NaN,72.0
2,NaN,102.3
3,NaN,60.9
4,NaN,148.0
5,NaN,47.0
6,NaN,50.0
7,NaN,63.4
8,NaN,48.0
9,NaN,30.0


In [237]:
# numeric
num_cols = ["price", "area", "n_bedrooms", "n_bathrooms", "n_floors", "front_width", "latitude", "longitude"]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors = "coerce")

# integer-like columns (nullable Int64)
for c in ["n_bedrooms", "n_bathrooms", "n_floors"]:
    if c in df.columns:
        df[c] = df[c].round().astype("Int64")

df.dtypes

property_type     object
price            float64
area             float64
n_bedrooms         Int64
n_bathrooms        Int64
n_floors           Int64
legal             object
interior          object
front_width      float64
latitude         float64
longitude        float64
address           object
district          object
city_province     object
dtype: object

In [238]:
# -------------------------------
# Remove old processed file (if exists)
# -------------------------------

import os

if os.path.exists(OUT_PATH):
    os.remove(OUT_PATH)
    print(f"Removed old file: {OUT_PATH}")
else:
    print("No existing file to remove.")

Removed old file: ../data/preprocessed/batdongsancomvn_preprocessed.csv


In [239]:
# -------------------------------
# Save cleaned dataset (overwrite)
# -------------------------------

os.makedirs(os.path.dirname(OUT_PATH), exist_ok = True)

df.to_csv(OUT_PATH, index = False, encoding = "utf-8")
print("Saved new clean file:", OUT_PATH)
print("Final shape:", df.shape)

Saved new clean file: ../data/preprocessed/batdongsancomvn_preprocessed.csv
Final shape: (8414, 14)


In [240]:
df.sample(5)
df.describe(include = "all").T.head(20)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
property_type,8414,4,Nhà riêng,2189,NaN,NaN,NaN,NaN,NaN,NaN,NaN
price,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
area,8379.0,NaN,NaN,NaN,113.989918,95.150479,1.0,60.0,83.8,128.0,991.0
n_bedrooms,6393.0,<NA>,<NA>,<NA>,4.36321,5.146761,1.0,2.0,3.0,5.0,97.0
n_bathrooms,6072.0,<NA>,<NA>,<NA>,4.132246,5.020741,1.0,2.0,3.0,5.0,97.0
n_floors,5210.0,<NA>,<NA>,<NA>,3.833205,1.730446,1.0,3.0,4.0,5.0,21.0
legal,7590,126,Sổ đỏ/ Sổ hồng,5280,NaN,NaN,NaN,NaN,NaN,NaN,NaN
interior,5175,339,Đầy đủ,2372,NaN,NaN,NaN,NaN,NaN,NaN,NaN
front_width,4353.0,NaN,NaN,NaN,7.103453,5.928738,2.0,4.2,5.0,8.0,149.0
latitude,8389.0,NaN,NaN,NaN,14.518398,4.778979,0.0,10.785719,10.849765,20.974672,22.332988


In [241]:
df.shape[0]

8414

In [242]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8414 entries, 0 to 8413
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   property_type  8414 non-null   object 
 1   price          0 non-null      float64
 2   area           8379 non-null   float64
 3   n_bedrooms     6393 non-null   Int64  
 4   n_bathrooms    6072 non-null   Int64  
 5   n_floors       5210 non-null   Int64  
 6   legal          7590 non-null   object 
 7   interior       5175 non-null   object 
 8   front_width    4353 non-null   float64
 9   latitude       8389 non-null   float64
 10  longitude      8389 non-null   float64
 11  address        8389 non-null   object 
 12  district       8413 non-null   object 
 13  city_province  8413 non-null   object 
dtypes: Int64(3), float64(5), object(6)
memory usage: 945.1+ KB
